In [7]:
from simplexlib.src.table import Table, V, Format, Simplex, pretty_value
from IPython.display import display_markdown


source = Table.straight(
    [2, 5, 3],
    V[2, 1, 2] <= 6,
    V[1, 2, 0] <= 6,
    V[0, 0.5, 1] <= 2,
    # expanded=True,
) >> min


Format.default_precision = 3


display_markdown(
    "### Исходная задача в канонической форме",
    f"${Format(source).target()}$",
    Format(source).system(),
    f"${Format(source).var_zero_constraint()}$",
    raw=True
)

### Исходная задача в канонической форме

$$F = 2x_1+5x_2+3x_3 \rightarrow min$$

$\begin{cases}2x_1+x_2+2x_3+x_4 = 6\\
x_1+2x_2+x_5 = 6\\
0.5x_2+x_3+x_6 = 2
\end{cases}$

$$x_0, x_1, x_2, x_3, x_4, x_5 ≥ 0$$

In [8]:
from src.cutting_plane import CuttingPlane, has_integer_solution

summary = CuttingPlane.resolve(source >> max)


for idx, result in enumerate(summary.history):
    display_markdown(
        f"### Шаг {idx + 1}",
        "### Исходная Simplex-таблица",
        Format(result.source, default_precision=3).table(),
        raw=True,
    )

    display_markdown(
        "### Конечная Simplex-таблица",
        Format(result.result, default_precision=3).table(),
        raw=True,
    )

    *vec, b = [pretty_value(val, 3) for val in CuttingPlane._infer_constraint(result.result).data]

    if has_integer_solution(result.result):
        display_markdown(
            "Найдено целочисленное решение",
            f"{Format(result.result).base_vars()}, {Format(result.result).free_vars()}",
            f"$F={pretty_value(result.result.F, 3)}$",
            raw=True,
        )
    else:
        display_markdown(
            "Добавим ограничение, задающееся следующими значениями:",
            f"$a=[{', '.join(vec)}], b={b}$",
            raw=True,
        )

### Шаг 1

### Исходная Simplex-таблица

|       |   $s_0$ |   $x_1$ |   $x_2$ |   $x_3$ |
|:------|--------:|--------:|--------:|--------:|
| $x_4$ |       6 |       2 |     1   |       2 |
| $x_5$ |       6 |       1 |     2   |       0 |
| $x_6$ |       2 |       0 |     0.5 |       1 |
| $F$   |      -0 |      -2 |    -5   |      -3 |

### Конечная Simplex-таблица

|       |   $s_0$ |   $x_4$ |   $x_5$ |   $x_6$ |
|:------|--------:|--------:|--------:|--------:|
| $x_1$ |    1    |   0.5   |    0    |   -1    |
| $x_2$ |    2.5  |  -0.25  |    0.5  |    0.5  |
| $x_3$ |    0.75 |   0.125 |   -0.25 |    0.75 |
| $F$   |   16.75 |   0.125 |    1.75 |    2.75 |

...

### Конечная Simplex-таблица

|       |   $s_0$ |   $x_8$ |   $x_7$ |   $x_6$ |
|:------|--------:|--------:|--------:|--------:|
| $x_1$ |       1 |    -1.5 |       1 |     0.5 |
| $x_2$ |       2 |     1   |       0 |    -1   |
| $x_3$ |       1 |    -0.5 |       0 |     1.5 |
| $x_5$ |       1 |    -0.5 |      -1 |     1.5 |
| $x_4$ |       0 |     3   |      -2 |    -3   |
| $F$   |      15 |     0.5 |       2 |     0.5 |

Найдено целочисленное решение

$x_1 = 1, x_2 = 2, x_3 = 1, x_5 = 1, x_4 = 0$, $x_8 = x_7 = x_6 = 0$

$F=15$